In [2]:

from bs4 import BeautifulSoup
import sqlite3
import requests
import pandas as pd
from random import randint
import time
from time import sleep
from typing import List
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score, f1_score

In [3]:
def sleepms(milliseconds: int):
    seconds = 0.001 * milliseconds
    sleep(seconds)
    
def random_wait():
    start = time.time()
    sleepms(randint(1, 350))

def getFakeUser():
    return {"User-Agent":"PostmanRuntime/7.29.0"}


def getRetHeaders():
    url='https://www.ret.co.il/m/91935/specs'
    page = requests.get(url, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    product=soup.find('div',class_='single-product-tab')
    data=product.findAll('ul','list-group list-group-horizontal')
    myheaders= ['מחיר מינימלי','מחיר מקסימלי']
    for alldata in data:
        myheaders.append(alldata.find('span',class_='text-dark font-weight-bold').text.strip())
    return myheaders



def getUrl(url):
    page = requests.get(url, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    MainClass=soup.find('ul', class_='nav nav-pills mb-2 justify-content-around justify-content-md-start col-xxl-10 offset-xxl-1 col-md-12 offset-md-0')
    ref=[]
    data=MainClass.find('a')
    technicalDataPage='https://www.ret.co.il'+data.get('href')
    return technicalDataPage

def getRetData(url):
    technicalDataPage=getUrl(url)
    page = requests.get(technicalDataPage, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    product=soup.find('div',class_='single-product-tab')
    dataLink=product.findAll('ul',class_='list-group list-group-horizontal')
    MinMaxP=MinMaxRetPrices(url)
    mydata=[MinMaxP[0],MinMaxP[1]]
    myheaders=[]
    for alldata in dataLink:
        # print(alldata.find('li',class_='list-group-item col-8 rounded-0 text-600').text.strip())
        try:
            mydata.append(alldata.find('li',class_='list-group-item col-8 rounded-0 text-600').text.strip())
        except:
             mydata.append('None')
    return mydata

def MinMaxRetPrices(url):
    page = requests.get(url, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    MainClass=soup.find('div',class_='')
    MainClass=soup.find('div',class_='card-body p-0 p-sm-3')
    price=MainClass.find('div',class_='price')
    if(price==None):
        return [None,None]
    else:
        price=price.text.strip()
        price=price.replace(',','')
        prices=price.split()
        MinPrice=prices[0]
        MaxPrice=prices[3]
        priceArray=MinPrice,MaxPrice
        return priceArray


In [4]:
def getRetToDataFrame(phonesURL):
    hrefLinks=[]
    allRetData=[]
    page = requests.get(phonesURL, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    MainClass=soup.findAll('a')
    for href in MainClass:
        if('/m/' in href.get('href')):
            hrefLinks.append('https://www.ret.co.il' + href.get('href'))
    for hrefLink in hrefLinks:
        allRetData.append(getRetData(hrefLink))
    return allRetData


In [5]:
def mainFunc(page_num_ret):
    head=getRetHeaders()
    url=f"https://www.ret.co.il/shop/cellphone?pageindex={page_num_ret}"
    data=getRetToDataFrame(url)
    df=pd.DataFrame(data,columns=head)
    return df

In [226]:
df_ret=mainFunc(1)
for i in range(2,10):
    df_ret=df_ret.append(mainFunc(i))
    random_wait()

C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\3648987791.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ret=df_ret.append(mainFunc(i))
C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\3648987791.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ret=df_ret.append(mainFunc(i))
C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\3648987791.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ret=df_ret.append(mainFunc(i))
C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\3648987791.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_ret=df_ret.append(mainFunc(i))
C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\3648987791.

In [212]:
def get_headers():
    url="https://www.zap.co.il/compmodels.aspx?modelid=1142728"
    page = requests.get(url, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    All_headers=['מחיר מינימלי','מחיר מקסימלי']
    headers=soup.find_all('div',class_="detailsRow")
    for header in headers:
        All_headers.append(header.find('div',class_="detailsRowTitletxt").text.strip())
    return All_headers


In [213]:
def get_properites_from_link(url):
    All_data=[]
    All_headers=[]
    page = requests.get(url, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    headers=soup.find_all('div',class_="detailsRow")
    titles = ['מחיר מינימלי','מחיר מקסימלי']
    prices = soup.find('div','PricesTxt').findAll('span') 
    if len(prices) == 2:
        All_data = [ prices[1].text, prices[0].text ]
    elif len(prices) == 1: 
        All_data = [ prices[0].text, prices[0].text ]
    else:
        All_data = [ None,None ]
    for header in headers:
        All_headers.append(header.find('div',class_="detailsRowTitletxt").text.strip())
        All_data.append(header.find('div',class_="detailsRowTxt").text.strip())
    return All_data

In [214]:
def get_data_from_zap(phonesURL):
    hrefLinks=[]
    all_zap_Data=[]
    Main_Url="https://www.zap.co.il"
    page = requests.get(phonesURL, headers = getFakeUser())
    soup= BeautifulSoup(page.content,'html.parser')
    links=soup.find_all('div',class_="MoreInfo")
    for link in links:
        hrefLinks.append(Main_Url+(link.find('a')['href']))
    for link in hrefLinks:
        all_zap_Data.append(get_properites_from_link(link))
    return all_zap_Data
    

In [215]:
def get_details_from_current_page(page_num_zap):
    url="https://www.zap.co.il/models.aspx?sog=e-cellphone&pageinfo={page_num_zap}"
    data=get_data_from_zap(url)
    df=pd.DataFrame(data,columns=get_headers())
    return df



Changing all the headers in the DataFrame from Hebrew to English

In [231]:

def change_CSV_File_Headers(df):
    df.rename(columns={"פוקוס אוטומטי": "Autofocus"}, inplace=True)
    df.rename(columns={"פלאש": "Flash"}, inplace=True)
    df.rename(columns={"מצלמה": "Camera"}, inplace=True)
    df.rename(columns={"מפעיל": "operator"}, inplace=True)
    df.rename(columns={"דור": "Generation"}, inplace=True)
    df.rename(columns={"סוג רשת": "Network kind"}, inplace=True)
    df.rename(columns={"כרטיס SIM": "SIM"},inplace=True)
    df.rename(columns={"רדיו FM": "FM radio"}, inplace=True)
    df.rename(columns={"גובה": "height"}, inplace=True)
    df.rename(columns={"רוחב": "width"}, inplace=True)
    df.rename(columns={"עובי": "thickness"}, inplace=True)
    df.rename(columns={"משקל": "wheight"}, inplace=True)
    df.rename(columns={"קיבולת סוללה": "Battery capacity"}, inplace=True)
    df.rename(columns={"סוללה נשלפת": "Removable battery"}, inplace=True)
    df.rename(columns={"רמקול חיצוני": "External speaker"}, inplace=True)
    df.rename(columns={"חיבור 3.5": "connection 3.5"},inplace=True)
    df.rename(columns={"מחיר מינימלי": "Minprice"}, inplace=True)
    df.rename(columns={"מחיר מקסימלי": "Maxprice"}, inplace=True)
    df.rename(columns={"יצרן": "Brand"}, inplace=True)
    df.rename(columns={"שנת הכרזה": "Declaration year"}, inplace=True)
    df.rename(columns={"מערכת הפעלה": "Operating System"}, inplace=True)
    df.rename(columns={"סדרה": "series"}, inplace=True)
    df.rename(columns={"מבנה המכשיר": "device structure"},inplace=True)
    df.rename(columns={"ממשק הפעלה": "operating interfaceo"}, inplace=True)
    df.rename(columns={"נפח אחסון פנימי": "Internal storage volume"}, inplace=True)
    df.rename(columns={"ערכת שבבים": "chipset"}, inplace=True)
    df.rename(columns={"צבעים": "colors"}, inplace=True)
    df.rename(columns={"טלפון כשר": "kosher phone"}, inplace=True)
    df.rename(columns={"רמת קרינה (SAR)": "radiation level"}, inplace=True)
    df.rename(columns={"טלפון למבוגרים": "Phone for adults"}, inplace=True)
    df.rename(columns={"מאיץ גרפי": "Graphics accelerator"}, inplace=True)
    df.rename(columns={"מהירות מעבד": "CPU speed"},inplace=True)
    df.rename(columns={"עמידות במים": "Water resistance"}, inplace=True)
    df.rename(columns={"קורא טביעת אצבע": "Fingerprint reader"},inplace=True)
    df.rename(columns={"גודל מסך": "screen size"}, inplace=True)
    df.rename(columns={"סוג מסך": "screen type"}, inplace=True)
    df.rename(columns={"קצב רענון": "refresh rate"}, inplace=True)
    df.rename(columns={"צפיפות פיקסלים": "Pixel density"}, inplace=True)
    df.rename(columns={"מפתח צמצם": "Aperture key"}, inplace=True)
    df.rename(columns={"מעבד": "Processor"}, inplace=True)
    df.rename(columns={"רזולוציה": "resolution"},inplace=True)
    df.rename(columns={"כרטיס זיכרון":"Memory Card"}, inplace=True)
    df.rename(columns={"זיכרון RAM": "RAM memory"}, inplace=True)
    df.rename(columns={"וידאו": "video"}, inplace=True)
    df.rename(columns={"קיבולת הסוללה": "battery capacity"}, inplace=True)
    df.rename(columns={"מספר ליבות": "number of cores"}, inplace=True)
    df.rename(columns={"מצלמה קדמית": "front camera"}, inplace=True)
    df.rename(columns={"טעינה אלחוטית": "wireless charging"}, inplace=True)
    df.to_csv("PhonesDF.csv",index=False)


In [217]:
df_zap=get_details_from_current_page(1)
for i in range(2,21):
    df_zap=df_zap.append(get_details_from_current_page(i))
    random_wait()

C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\2310696407.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_zap=df_zap.append(get_details_from_current_page(i))
C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\2310696407.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_zap=df_zap.append(get_details_from_current_page(i))
C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\2310696407.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_zap=df_zap.append(get_details_from_current_page(i))
C:\Users\eranb\AppData\Local\Temp\ipykernel_4840\2310696407.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_zap=df_zap.append(get_detail

Saving the first DataFrame before cleaning

In [ ]:
def SaveDataFrame(df):
    df=df_zap.append(df_ret)
    df.to_csv('PhonesDF.csv',header=True)
    df=pd.read_csv('PhonesDF.csv')
    change_CSV_File_Headers(df)
    df=pd.read_csv('PhonesDF.csv')
    df


Function to clean all the Data Frame None price columns

In [28]:
def CleanTheDF(df):
    df=df.drop_duplicates()
    df.dropna(axis=0, how='all')# if all the values are None remove this row
    df=df.dropna(subset=['Minprice','Maxprice'],how ='all')
    df.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
    df.to_csv('SmallPhonesDF.csv')
    return df

In [120]:
df=pd.read_csv('PhonesDF.csv')
Smalldf=CleanTheDF(df)
Smalldf


,Minprice,Maxprice,Brand,Operating System,Declaration year,device structure,series,Smartphone,radiation level,operating interfaceo,kosher phone,colors,Phone for adults,Internal storage volume,chipset,Processor,number of cores,CPU speed,Graphics accelerator,RAM memory,Memory Card,Water resistance,Fingerprint reader,screen size,screen type,resolution,Pixel density,refresh rate,SIM,Network kind,Generation,operator,WiFi,Bluetooth,IR,GPS,NFC,Camera,Flash,Autofocus,Aperture key,video,front camera,connection 3.5,External speaker,FM radio,height,width,thickness,wheight,battery capacity,Removable battery,wireless charging,Dual-SIM
0,"3,458","4,649",Samsung,Android,2022,פאבלט,Galaxy S22,סמארטפון,1.05 W/kg,מסך מגע,יעודכן בקרוב,"אדום , ירוק , לבן , שחור , תכלת",לא רלוונטי,256GB,Snapdragon 8 Gen 1,Cortex-X2 Cortex-A710 Cortex-A510,4 + 3 + 1 ליבות,1700 2400 3000 MHz,Adreno 730,12GB,ללא,IP68,כולל,6.8 אינטש,Dynamic AMOLED 2X,1440x3088 פיקסלים,500 ppi,120 Hz,eSIM,"5G , LTE",4,"רמי לוי , פלאפון , הוט מובייל , גולן טלקום , ס...",כולל,5.2,לא זמין,כולל,כולל,108 + 10 + 10 + 12 מגה פיקסל,לד אחד,לא זמין,1.8 4.9 2.4 2.2 /f,4320p@24fps,40 מגה פיקסל,ללא,זוג רמקולים,יעודכן בקרוב,"163.3 מ""מ","77.9 מ""מ","8.9 מ""מ",228 גרם,"5,000 mAh",לא,כולל,NaN
1,"2,774","3,499",Apple,iOS,2021,פאבלט,iPhone 13,סמארטפון,יעודכן בקרוב,מסך מגע,יעודכן בקרוב,"אדום , ורוד , כחול , שחור",יעודכן בקרוב,128GB,Apple A15 Bionic,יעודכן בקרוב,2 + 4 ליבות,יעודכן בקרוב,Apple GPU,4GB,ללא,IP68,ללא,6.1 אינטש,Super Retina,1170x2532 פיקסלים,460 ppi,לא זמין,"SIM אחד , eSIM",5G,5,יעודכן בקרוב,כולל,5.0,לא כולל,כולל,כולל,12 + 12 מגה פיקסלים,שני לדים,כולל,1.6 2.4 /f,2160p@60fps,12 מגה פיקסל,ללא,זוג רמקולים,ללא רדיו,"146.7 מ""מ","71.5 מ""מ","7.7 מ""מ",174 גרם,לא זמין,לא,כולל,NaN
2,"5,795","6,699",Apple,iOS,2022,פאבלט,iPhone 14 Pro Max,סמארטפון,לא זמין,מסך מגע,לא רלוונטי,"זהב , כסוף , סגול , שחור",לא רלוונטי,256GB,Apple A16 Bionic,יעודכן בקרוב,יעודכן בקרוב,יעודכן בקרוב,Apple GPU,6GB,ללא,IP68,לא זמין,6.7 אינטש,LTPO Super Retina XDR OLED,1290x2796 פיקסלים,460 ppi,120 Hz,Dual SIM אופציונלי,"5G , LTE",4,"רמי לוי , פלאפון , הוט מובייל , גולן טלקום , ס...",כולל,5.3,לא זמין,כולל,כולל,48 + 12 + 12 צגה פיקסל,שני לדים,לא זמין,1.8 2.8 2.2 /f,2160p@60fps,12 מגה פיקסל,ללא,זוג רמקולים,ללא רדיו,"160.7 מ""מ","77.6 מ""מ","7.9 מ""מ",240 גרם,לא זמין,לא,כולל,NaN
3,"1,510","1,949",Samsung,Android,2022,פאבלט,Galaxy A73,סמארטפון,1.28 W/kg,מסך מגע,יעודכן בקרוב,"אפור , לבן",יעודכן בקרוב,128GB,Snapdragon 778G,Kryo 670 + Kryo 670,4 + 4 ליבות,1800 2400 MHz,Adreno 642L,8GB,microSDXC,IP67,כולל,6.7 אינטש,Super AMOLED Plus,1080x2400 פיקסלים,393 ppi,120 Hz,Dual SIM אופציונלי,"5G , LTE",4,"רמי לוי , פלאפון , הוט מובייל , גולן טלקום , ס...",כולל,5.0,לא זמין,כולל,אופציונאלי,108 + 12 + 5 + 5 מגה פיקסל,לד אחד,לא זמין,1.8 2.2 2.4 2.4 /f,2160p@30fps,32 מגה פיקסל,יעודכן בקרוב,יחיד,יעודכן בקרוב,"163.7 מ""מ","76.1 מ""מ","7.6 מ""מ",181 גרם,"5,000 mAh",לא,ללא,NaN
4,"3,193","3,965",Apple,iOS,2022,פאבלט,iPhone 14,סמארטפון,לא זמין,מסך מגע,יעודכן בקרוב,"אדום , כחול , סגול",יעודכן בקרוב,128GB,Apple A15 Bionic,יעודכן בקרוב,4 + 2 ליבות,1820 3230 MHz,Apple GPU,4GB,ללא,IP68,לא זמין,6.1 אינטש,Super Retina XDR OLED,1170x2532 פיקסלים,460 ppi,לא זמין,Dual SIM אופציונלי,"5G , LTE",4,"רמי לוי , פלאפון , הוט מובייל , גולן טלקום , ס...",כולל,5.3,לא זמין,כולל,כולל,12 + 12 מגה פיקסלים,שני לדים,לא זמין,1.5 2.4 /f,2160p@60fps,12 מגה פיקסל,ללא,זוג רמקולים,ללא רדיו,"146.7 מ""מ","71.5 מ""מ","7.8 מ""מ",172 גרם,לא זמין,לא,כולל,NaN
5,"2,100","2,999",Samsung,Android,2022,פאבלט,Galaxy S22,סמארטפון,1.21 W/kg,מסך מגע,לא רלוונטי,"ורוד זהב , ירוק , לבן , שחור , תכלת",לא רלוונטי,128GB,Snapdragon 8 Gen 1,Cortex-X2 Cortex-A710 Cortex-A510,4 + 3 + 1 ליבות,1700 2400 3000 MHz,Adreno 730,8GB,ללא,IP68,כולל,6.1 אינטש,Dynamic AMOLED 2X,1080x2340 פיקסלים,425 ppi,120 Hz,Dual SIM,"5G , LTE",4,"רמי לוי , פלאפון , הוט מובייל , גולן טלקום , ס...",כולל,5.2,לא זמין,כולל,כולל,50 + 10 + 12 מגה פיקסל,לד אחד,לא זמין,1.8 2.4 2.2 /f,4320p@24fps,10 מגה פיקסל,ללא,זוג רמקולים,ללא רדיו,"146 מ""מ","70.6 

Cleaning and converting to Numeric

In [121]:
def cleaning_to_numeric(df):

    df['radiation level']=pd.to_numeric(df['radiation level'].str.replace('W/kg','').str.strip(),errors='coerce')
    df['height']=pd.to_numeric(df['height'].str.replace('מ"מ','').str.strip(),errors='coerce')
    df['Internal storage volume']=pd.to_numeric(df['Internal storage volume'].str.replace('GB','').str.replace('1TB','1000').str.strip(),errors='coerce')
    df['RAM memory']=pd.to_numeric(df['RAM memory'].str.replace('GB','').str.strip(),errors='coerce')
    df['width']=pd.to_numeric(df['width'].str.replace('מ"מ','').str.strip(),errors='coerce')
    df['thickness']=pd.to_numeric(df['thickness'].str.replace('מ"מ','').str.strip(),errors='coerce')
    df['wheight']=pd.to_numeric(df['wheight'].str.replace('גרם','').str.strip(),errors='coerce')
    df['battery capacity']=pd.to_numeric(df['battery capacity'].str.replace('mAh','').str.strip(),errors='coerce')
    df['refresh rate']=pd.to_numeric(df['refresh rate'].str.replace('Hz','').str.strip(),errors='coerce')
    df['Pixel density']=pd.to_numeric(df['Pixel density'].str.replace('ppi','').str.strip(),errors='coerce')
    df['number of cores']=pd.to_numeric(df['number of cores'].str.replace('ליבות','').str.strip(),errors='coerce')
    df['screen size']=pd.to_numeric(df['screen size'].str.replace('אינטש','').str.strip(),errors='coerce')
    df['Network kind']=pd.to_numeric(df['Network kind'].str.replace('G','').str.strip(),errors='coerce')
    df['front camera']=pd.to_numeric(df['front camera'].str.replace('מגה פיקסל','').str.strip(),errors='coerce')
    df['Flash']=pd.to_numeric(df['Flash'].str.replace('לדים','').str.strip(),errors='coerce')
    #df['resolution']=pd.to_numeric(df['resolution'].str.replace('פיקסלים','').str.strip(),errors='coerce')
    df['Camera']=pd.to_numeric(df['Camera'].str.replace('מגה פיקסלים','').str.strip(),errors='coerce')
    df['CPU speed']=pd.to_numeric(df['CPU speed'].str.replace('MHz','').str.strip(),errors='coerce')

    return df

In [122]:
df_WithNone = Smalldf.copy()
for col in df_WithNone:
    df_WithNone.loc[(df_WithNone[col] == 'לא זמין') |(df_WithNone[col] == 'יעודכן בקרוב'),col]=None
df_WithNone=cleaning_to_numeric(df_WithNone)

df_WithNone["Autofocus"] =  np.where(df_WithNone['Autofocus']  == "כולל" , 1 , 0)
df_WithNone["connection 3.5"] =  np.where(df_WithNone['connection 3.5']  == "ללא" , 0 , 1)
df_WithNone["Dual-SIM"] =  np.where(df_WithNone['Dual-SIM']  == "ללא" , 0 , 1)
df_WithNone["Memory Card"] =  np.where(df_WithNone['Memory Card']  == "ללא" , 0 , 1)
df_WithNone["Fingerprint reader"] =  np.where(df_WithNone['Fingerprint reader']  == "ללא" , 0 , 1)
df_WithNone["WiFi"] =  np.where(df_WithNone['WiFi']  == "כולל" , 1 , 0)
df_WithNone["IR"] =  np.where(df_WithNone['IR']  == "לא כולל" , 0 , 1)
df_WithNone["GPS"] =  np.where(df_WithNone['GPS']  == "כולל" , 1 , 0)
df_WithNone["NFC"] =  np.where(df_WithNone['NFC']  == "כולל" , 1 , 0)
df_WithNone["FM radio"] =  np.where(df_WithNone['FM radio']  == "ללא רדיו" , 0 , 1)
df_WithNone["Removable battery"] =  np.where(df_WithNone['Removable battery']  == "לא" , 0 , 1)
df_WithNone["wireless charging"] =  np.where(df_WithNone['wireless charging']  == "כולל" , 1 , 0)
df_WithNone["External speaker"] =  np.where(df_WithNone['External speaker']  == "ללא" , 0 , 1)
df_WithNone

In [123]:
df_WithNone

,Minprice,Maxprice,Brand,Operating System,Declaration year,device structure,series,Smartphone,radiation level,operating interfaceo,kosher phone,colors,Phone for adults,Internal storage volume,chipset,Processor,number of cores,CPU speed,Graphics accelerator,RAM memory,Memory Card,Water resistance,Fingerprint reader,screen size,screen type,resolution,Pixel density,refresh rate,SIM,Network kind,Generation,operator,WiFi,Bluetooth,IR,GPS,NFC,Camera,Flash,Autofocus,Aperture key,video,front camera,connection 3.5,External speaker,FM radio,height,width,thickness,wheight,battery capacity,Removable battery,wireless charging,Dual-SIM
0,"3,458","4,649",Samsung,Android,2022,פאבלט,Galaxy S22,סמארטפון,1.050,מסך מגע,None,"אדום , ירוק , לבן , שחור , תכלת",לא רלוונטי,256.0,Snapdragon 8 Gen 1,Cortex-X2 Cortex-A710 Cortex-A510,NaN,NaN,Adreno 730,12.0,0,IP68,1,6.80,Dynamic AMOLED 2X,1440x3088 פיקסלים,500.0,120.0,eSIM,NaN,4,"רמי לוי , פלאפון , הוט מובייל , גולן טלקום , ס...",1,5.2,1,1,1,NaN,NaN,0,1.8 4.9 2.4 2.2 /f,4320p@24fps,40.0,0,1,1,163.3,77.9,8.90,228.0,NaN,0,1,1
1,"2,774","3,499",Apple,iOS,2021,פאבלט,iPhone 13,סמארטפון,NaN,מסך מגע,None,"אדום , ורוד , כחול , שחור",None,128.0,Apple A15 Bionic,None,NaN,NaN,Apple GPU,4.0,0,IP68,0,6.10,Super Retina,1170x2532 פיקסלים,460.0,NaN,"SIM אחד , eSIM",5.0,5,None,1,5.0,0,1,1,NaN,NaN,1,1.6 2.4 /f,2160p@60fps,12.0,0,1,0,146.7,71.5,7.70,174.0,NaN,0,1,1
2,"5,795","6,699",Apple,iOS,2022,פאבלט,iPhone 14 Pro Max,סמארטפון,NaN,מסך מגע,לא רלוונטי,"זהב , כסוף , סגול , שחור",לא רלוונטי,256.0,Apple A16 Bionic,None,NaN,NaN,Apple GPU,6.0,0,IP68,1,6.70,LTPO Super Retina XDR OLED,1290x2796 פיקסלים,460.0,120.0,Dual SIM אופציונלי,NaN,4,"רמי לוי , פלאפון , הוט מובייל , גולן טלקום , ס...",1,5.3,1,1,1,NaN,NaN,0,1.8 2.8 2.2 /f,2160p@60fps,12.0,0,1,0,160.7,77.6,7.90,240.0,NaN,0,1,1
3,"1,510","1,949",Samsung,Android,2022,פאבלט,Galaxy A73,סמארטפון,1.280,מסך מגע,None,"אפור , לבן",None,128.0,Snapdragon 778G,Kryo 670 + Kryo 670,NaN,NaN,Adreno 642L,8.0,1,IP67,1,6.70,Super AMOLED Plus,1080x2400 פיקסלים,393.0,120.0,Dual SIM אופציונלי,NaN,4,"רמי לוי , פלאפון , הוט מובייל , גולן טלקום , ס...",1,5.0,1,1,0,NaN,NaN,0,1.8 2.2 2.4 2.4 /f,2160p@30fps,32.0,1,1,1,163.7,76.1,7.60,181.0,NaN,0,0,1
4,"3,193","3,965",Apple,iOS,2022,פאבלט,iPhone 14,סמארטפון,NaN,מסך מגע,None,"אדום , כחול , סגול",None,128.0,Apple A15 Bionic,None,NaN,NaN,Apple GPU,4.0,0,IP68,1,6.10,Super Retina XDR OLED,1170x2532 פיקסלים,460.0,NaN,Dual SIM אופציונלי,NaN,4,"רמי לוי , פלאפון , הוט מובייל , גולן טלקום , ס...",1,5.3,1,1,1,NaN,NaN,0,1.5 2.4 /f,2160p@60fps,12.0,0,1,0,146.7,71.5,7.80,172.0,NaN,0,1,1
5,"2,100","2,999",Samsung,Android,2022,פאבלט,Galaxy S22,סמארטפון,1.210,מסך מגע,לא רלוונטי,"ורוד זהב , ירוק , לבן , שחור , תכלת",לא רלוונטי,128.0,Snapdragon 8 Gen 1,Cortex-X2 Cortex-A710 Cortex-A510,NaN,NaN,Adreno 730,8.0,0,IP68,1,6.10,Dynamic AMOLED 2X,1080x2340 פיקסלים,425.0,120.0,Dual SIM,NaN,4,"רמי לוי , פלאפון , הוט מובייל , גולן טלקום , ס...",1,5.2,1,1,1,NaN,NaN,0,1.8 2.4 2.2 /f,4320p@24fps,10.0,0,1,0,146.0,70.6,7.60,167.0,NaN,0,1,1
6,"2,285","3,699",Apple,iOS,2020,פאבלט,iPhone 12,סמארטפון,NaN,מסך מגע,לא רלוונטי,"אדום , ירוק , כחול , לבן , שחור",לא רלוונטי,128.0,Apple A14,None,8.0,NaN,None,NaN,0,IP68,0,6.10,"OLED , Super Retina",1170x2532 פיקסלים,460.0,NaN,"SIM אחד , eSIM",5.0,5,"רמי לוי , פלאפון , הוט מובייל , גולן טלקום , ס...",1,5.0,0,1,1,NaN,4.0,1,1.6 2.4 /f,2160p@60fps,12.0,0,1,0,146.1,71.5,7.40,164.0,NaN,0,1,1
7,"1,677","2,400",Samsung,Android,2022,פאבלט,Galaxy S21,סמארטפון,0.950,מסך מגע,לא רלוונטי,"גרפיט , ירוק זית , לבן , לבנדר",לא רלוונטי,128.0,None,Snapdragon 888,NaN,NaN,Adreno 660,8.0,0,IP68,1,6.40,Dynamic AMOLED 2X,1080x2400 פיקסלים,411.0,120.0,Dual SIM אופציונלי,NaN,4,"רמי לוי , פלאפון , הוט מובייל , גולן טלקום , ס...",1,כולל,1,1,1,NaN,NaN,0,1.8 2.4 2.2 /f,2160p@60fps,32.0,0,1,0,155.7,74.5,7.90,177.0,NaN,0,1,1
8,668,"1,079",Xiaomi,Android,2022,פאבלט,Redmi Note 11,סמארטפון,NaN,מסך מגע,None,"אפור , כחול , לבן",None,128.0,Snapdragon 680,Kryo 265 + Kryo 265,NaN,NaN,Adreno

In [119]:
df_WithNone['resolution']

0     NaN
1     NaN
2     NaN
3     NaN
4     NaN
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
10    NaN
11    NaN
12    NaN
13    NaN
14    NaN
15    NaN
16    NaN
17    NaN
18    NaN
19    NaN
20    NaN
21    NaN
22    NaN
23    NaN
33    NaN
48    NaN
101   NaN
107   NaN
113   NaN
480   NaN
481   NaN
482   NaN
483   NaN
484   NaN
485   NaN
486   NaN
487   NaN
488   NaN
489   NaN
490   NaN
491   NaN
492   NaN
493   NaN
494   NaN
495   NaN
496   NaN
497   NaN
498   NaN
499   NaN
500   NaN
501   NaN
502   NaN
503   NaN
504   NaN
505   NaN
506   NaN
507   NaN
508   NaN
509   NaN
510   NaN
511   NaN
512   NaN
513   NaN
514   NaN
515   NaN
516   NaN
517   NaN
518   NaN
519   NaN
520   NaN
521   NaN
522   NaN
523   NaN
524   NaN
525   NaN
526   NaN
527   NaN
528   NaN
529   NaN
530   NaN
531   NaN
532   NaN
533   NaN
534   NaN
535   NaN
536   NaN
537   NaN
538   NaN
539   NaN
540   NaN
541   NaN
542   NaN
543   NaN
544   NaN
545   NaN
546   NaN
547   NaN
548   NaN
549   NaN
550   NaN
